In [223]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import tensorflow as tf
import keras.models
#from keras.models import Sequential
from keras.layers import Conv2D , MaxPool2D ,Flatten , Dense, Dropout
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from PIL import Image
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D , BatchNormalization;
#from models import custom_convnet
from keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import classification_report

In [257]:
job=pd.read_csv(r"C:\Users\REAL\Documents\Datasets\NLP\fake_job_postings.csv", encoding='latin1')

In [258]:
df2=job[job['benefits'].isna()]

In [259]:
df2.fraudulent.value_counts()

0    6846
1     364
Name: fraudulent, dtype: int64

In [260]:
job['x'] = job['salary_range'].apply(str)+' '+job['company_profile'].apply(str)+' '+job['has_company_logo'].apply(str)+' '+job['location'].apply(str)+' '+job['benefits'].apply(str)

In [261]:
job = job.loc[: , ['x','fraudulent']]

In [262]:
train,test = train_test_split(job)

In [263]:
df_temp = train[train.fraudulent==1]
train = pd.concat([train,df_temp,df_temp,df_temp])

In [264]:
x_train=train.x
y_train = train.loc[:,['fraudulent']]

x_test=test.x
y_test = test.loc[:,['fraudulent']]

In [265]:
from sklearn.model_selection import train_test_split
# x_train , x_test , y_train, y_test = train_test_split(job['x'] , job['fraudulent'] , test_size = .2)

In [266]:
# # Assuming data is your dataset
# from sklearn.model_selection import train_test_split

# # Adjust validation_split as needed
# X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42, shuffle=True)


In [267]:
y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

In [268]:
max_num_words=10000             ## from the entire corpus select 10000 words
seq_len=50                      ## how many words out of 10000 you wish to take from 1 doc
embedding_size=100

In [269]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [270]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(job.x)
x_train = tokenizer.texts_to_sequences(x_train)

x_test = tokenizer.texts_to_sequences(x_test)

In [271]:
x_train = pad_sequences(x_train , maxlen = seq_len)

x_test = pad_sequences(x_test , maxlen = seq_len)

In [272]:
x_test.shape

(4470, 50)

In [273]:
model = Sequential()
model.add(Embedding(input_dim = max_num_words,
                   input_length = seq_len,
                   output_dim = embedding_size))

In [274]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from keras.layers import Bidirectional

In [275]:
model.add(Bidirectional(LSTM(64)))
model.add(Dense(2, activation = 'softmax'))  #o/p layer
adam = Adam(learning_rate = 0.001)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [276]:
model.fit(x_train,y_train,batch_size=32,epochs =5)

Epoch 1/5
482/482 [==============================] - 23s 37ms/step - loss: 0.1676 - accuracy: 0.9287
Epoch 2/5
482/482 [==============================] - 18s 37ms/step - loss: 0.0683 - accuracy: 0.9712
Epoch 3/5
482/482 [==============================] - 18s 36ms/step - loss: 0.0521 - accuracy: 0.9779
Epoch 4/5
482/482 [==============================] - 17s 36ms/step - loss: 0.0457 - accuracy: 0.9792
Epoch 5/5
482/482 [==============================] - 17s 36ms/step - loss: 0.0456 - accuracy: 0.9795


In [277]:
pred = model.predict(x_test)
pred

140/140 [==============================] - 2s 10ms/step


array([[9.9996746e-01, 3.2586770e-05],
       [9.8305744e-01, 1.6942551e-02],
       [9.9999607e-01, 3.9498550e-06],
       ...,
       [9.9999893e-01, 1.0468679e-06],
       [9.9999976e-01, 2.3417908e-07],
       [9.4228494e-01, 5.7715051e-02]], dtype=float32)

In [278]:
pred_classes= pred.argmax(axis = 1)
#y_test= y_test.argmax(axis = 1)

In [279]:
tab = confusion_matrix(y_test , pred_classes)
tab

array([[4165,  101],
       [  45,  159]], dtype=int64)

In [280]:
from sklearn.metrics import confusion_matrix,accuracy_score


In [281]:
accuracy_score(y_test , pred_classes)*100

96.7337807606264

In [282]:
print(classification_report(y_test,pred_classes))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4266
           1       0.61      0.78      0.69       204

    accuracy                           0.97      4470
   macro avg       0.80      0.88      0.83      4470
weighted avg       0.97      0.97      0.97      4470



In [283]:
model.save('fake_job_LSTM.h5')

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
